In [1]:
import pickle
from pathlib import Path

import joblib
import keras
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

from utils import *

2024-04-02 13:18:12.625370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 13:18:12.625430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 13:18:12.626247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 13:18:12.631752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pc = ProjectConfig()
MODEL_DIR = pc.project_root_dir.joinpath("src").joinpath("model_fine_tuned_saves")
MODEL_NAME = "efficientnet_v2m"
MODEL_PATH = MODEL_DIR.joinpath(MODEL_NAME + ".h5")
CLASS_LIST_PATH = pc.project_root_dir.joinpath("src").joinpath("class_list.lzma")
TRAIN_DATA = pc.data_root_dir.joinpath("training_data_71")

No configuration file found. Using defaults.


In [3]:
if CLASS_LIST_PATH.exists():
    classes = joblib.load(CLASS_LIST_PATH)
else:
    print("No class list found, please run the training script first")

In [4]:
if TRAIN_DATA.exists() and TRAIN_DATA.is_dir():
    print("Test data found, loading")
    train_data = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DATA,
        labels="inferred",
        label_mode="int",
        batch_size=32,
        image_size=(224, 224),
        shuffle=False,
    )
else:
    print("No test data found, where did it go?")

Test data found, loading
Found 31227 files belonging to 71 classes.


2024-04-02 13:19:17.692734: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-02 13:19:17.697778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-02 13:19:17.698104: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-02 13:19:17.699121: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-02 13:19:17.699445: I external/local_xla/xla/stream_executor

In [5]:
if MODEL_PATH.exists():
    model = tf.keras.models.load_model(MODEL_PATH)
    model.trainable = False
    print("Model loaded from disk")
else:
    print("Model not found at path:", MODEL_PATH)

Model loaded from disk


In [6]:
training_predictions_file = pc.data_root_dir.joinpath("training_data_71_predictions.lzma")
if training_predictions_file.exists():
    print("Predictions loaded from disk")
    training_predictions = joblib.load(training_predictions_file)
else:
    print("Predictions not found at path:", training_predictions_file)
    print("Predicting")
    predictions = model.predict(train_data)
    joblib.dump(predictions, training_predictions_file)

Predictions not found at path: /tf/notebooks/data/training_data_71_predictions.lzma
Predicting


2024-04-02 13:20:24.332826: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


976/976 [==============================] - 88s 85ms/step


In [7]:
predicted_classes = np.argmax(predictions, axis=-1)

In [8]:
file_paths = train_data.file_paths
class_names = train_data.class_names
true_labels = []
for images, true_label in train_data:
    true_labels.extend(true_label.numpy())
true_class_labels = [class_names[i] for i in true_labels]
predicted_class_labels = [class_names[i] for i in predicted_classes]


In [12]:
df = pl.DataFrame(zip(file_paths, true_class_labels, predicted_class_labels), schema=[("file_path", pl.Utf8), ("true_class_labels", pl.Utf8), ("predicted_label", pl.Utf8)])
mispredicted_df = df.filter(pl.col("true_class_labels") != pl.col("predicted_label")).sort("true_class_labels")
predicted_df = df.filter(pl.col("true_class_labels") == pl.col("predicted_label")).sort("true_class_labels")



In [14]:
print(mispredicted_df.height)
print(predicted_df.height)

3855
27372


In [15]:
df.write_parquet(pc.data_root_dir.joinpath("training_data_71_prediction_results.parquet"))
df.write_csv(pc.data_root_dir.joinpath("training_data_71_prediction_results.csv"))